# Neural Networks

In this notebook, we revisit what is a Neural Network, and how to train a Neural Network with PyTorch.

In this notebook, we will also be using the `torchvision` module as it provides additional functionalities in Computer Vision.

In [1]:
import torch
from torch import optim, nn, Tensor
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import v2

from __future__ import annotations

## Create a Neural Network

In [2]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # First 2D convolutional layer, taking in 3 input channel (RGB image),
      # outputting 32 convolutional features, with a square kernel size of 3
      self.conv1 = nn.Conv2d(3, 32, 3, 1)
      # Second 2D convolutional layer, taking in the 32 input layers,
      # outputting 64 convolutional features, with a square kernel size of 3
      self.conv2 = nn.Conv2d(32, 64, 3, 1)

      # Designed to ensure that adjacent pixels are either all 0s or all active
      # with an input probability
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)

      # First fully connected layer
      self.fc1 = nn.Linear(12544, 1024)
      # Second fully connected layer that outputs our 100 labels
      self.fc2 = nn.Linear(1024, 100)

    def forward(self, x: Tensor):
      # Pass data through conv1
      x = self.conv1(x)
      # Use the rectified-linear activation function over x
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      # Run max pooling over x
      x = F.max_pool2d(x, 2)
      # Pass data through dropout1
      x = self.dropout1(x)
      # Flatten x with start_dim=1
      x = torch.flatten(x, 1)
      # Pass data through ``fc1``
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # Apply softmax to x
      # `softmax` opertion is often not needed as Criterion has it built-in
      # x = F.log_softmax(x, dim=1)
      return x

## Build `Dataset` and `DataLoader`

`torch.utils.data` provides 3 utilities classes to help you manage your data.

1. `Dataset`: read data from disk & performs data augmentation
2. `Sampler`: sample a batch of indices from the dataset (optional, `DataLoader` has default `Sampler`)
3. `DataLoader`: collate multiple data points to a batch (use built-in `DataLoader` in most cases)

In many cases, you will need to write your own `Dataset` to read your own data.

`torchvision` also has some pre-defined dataset classes like `torchvision.datasets.ImageFolder`.

In [3]:
class DataSet(Dataset):
    def __init__(self):
        ...
    def __getitem__(self, index: int):
        ...
    def __getitems__(self, indices: list[int]):
        ...
    def __len__(self):
        ...

In [4]:
transform = v2.Compose(
    [
        # Convert image read from the disk (in PIL.Image or OpenCV) to Tensor
        v2.ToTensor(),
        # Normalize the image to a normal distribution (with a mean of 0 and a standard deviation of 1)
        # note thad different dataset may have their own `mean` and `std`
        v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        # Optional data augmentations 
        v2.RandomResizedCrop((32, 32), antialias=True),
        v2.RandomVerticalFlip(),
        v2.RandomHorizontalFlip(),
        v2.ColorJitter(),
    ]
)

train_data = datasets.CIFAR100("cifar100", transform=transform, download=True)
val_data = datasets.CIFAR100("cifar100", transform=transform, train=False, download=True)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


100%|██████████| 169001437/169001437 [00:14<00:00, 11287195.28it/s]


Extracting cifar100/cifar-100-python.tar.gz to cifar100
Files already downloaded and verified


In [5]:
model = Net()
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader))

In [6]:
def train_step(input: Tensor, label: Tensor):
    output = model(input)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()
    return loss, output

def train_epoch(dataloader):
    for i, (input, label) in enumerate(dataloader):
        loss, output = train_step(input, label)
        if i % 100 == 0:
            print(f"Loss: {loss.item()}")

train_epoch(train_loader)


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Loss: 4.599312782287598
Loss: 4.631166934967041
Loss: 4.621702671051025
Loss: 4.526247024536133
Loss: 4.630471229553223
Loss: 4.610811710357666
Loss: 4.499372482299805
Loss: 4.599653720855713
Loss: 4.548818588256836
Loss: 4.4871826171875
Loss: 4.45155143737793
Loss: 4.355360984802246
Loss: 3.9637246131896973
Loss: 4.619658470153809
Loss: 4.4346771240234375
Loss: 4.777684211730957
Loss: 4.345715045928955
Loss: 3.9502010345458984
Loss: 4.4586029052734375
Loss: 4.400957107543945
Loss: 4.498456954956055
Loss: 4.248757362365723
Loss: 4.136300563812256
Loss: 4.589773654937744
Loss: 4.244424343109131
Loss: 4.136141300201416
Loss: 4.372920513153076
Loss: 4.847926139831543
Loss: 4.069480895996094
Loss: 3.8857955932617188
Loss: 4.184459686279297
Loss: 4.309844493865967
Loss: 4.668144226074219
Loss: 4.744712829589844
Loss: 3.9085421562194824
Loss: 4.07913875579834
Loss: 3.8500194549560547
Loss: 4.152214527130127
Loss: 4.617140769958496
Loss: 4.724593162536621
Loss: 4.0057597160339355
Loss: 4.1569

In [7]:
@torch.no_grad()
def evaluate_step(input, label):
    output = model(input)
    loss = criterion(output, label)
    return loss, output

def evaluate_epoch(dataloader):
    for i, (input, label) in enumerate(dataloader):
        loss, output = evaluate_step(input, label)
        pred = output.argmax(dim=1, keepdim=True)
        accuracy = pred.eq(label.view_as(pred)).sum().item() / len(label)
        if i % 100 == 0:
            print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

evaluate_epoch(val_loader)

Loss: 4.295488357543945, Accuracy: 0.0
Loss: 4.139092445373535, Accuracy: 0.0
Loss: 4.592042922973633, Accuracy: 0.0
Loss: 3.7268476486206055, Accuracy: 0.125
Loss: 3.7312283515930176, Accuracy: 0.0
Loss: 4.1175856590271, Accuracy: 0.125
Loss: 4.0747809410095215, Accuracy: 0.0
Loss: 4.104453086853027, Accuracy: 0.0
Loss: 3.6606931686401367, Accuracy: 0.125
Loss: 4.5535569190979, Accuracy: 0.0
Loss: 4.485076904296875, Accuracy: 0.0
Loss: 4.329070091247559, Accuracy: 0.0
Loss: 4.460677623748779, Accuracy: 0.0


## Save your trained model

In [8]:
torch.save(model.state_dict(), "model.pth")